Use eBird API to Create REF table in funData database

In [1]:
import requests as req
import json
import pandas as pd
import numpy as np
import pyodbc
import sqlalchemy
import urllib

In [8]:
apiToken = 'qfosq81fn99u'
headers_dict = {'X-eBirdApiToken' : apiToken}
resp = req.get(f'https://api.ebird.org/v2/ref/taxonomy/ebird', headers=headers_dict)
print(len(resp.text))

speciesList = []

cntr = 0
for line in resp.text.split('\n'):

    entries = line.split(',')   

    if cntr == 0:
        cntr +=1
        continue
  
    if len(entries) <= 1:
        continue 

    speciesList.append([entries[2], entries[1], entries[0], entries[6], entries[8], entries[9], entries[10], entries[14] ])
    
    cntr+=1



arr = np.array(speciesList)
df = pd.DataFrame(data=arr, columns=['Species_Code', 'Species', 'Scientific_Name', 'Sci_Name_Codes', 'Order', 'Family_Com_Name', 'Family_Sci_Name', 'Family_Code'])
df['Species'] = df['Species'].apply(lambda x: x.title())
df['Order'] = df['Order'].apply(lambda x: x.replace('"', '').title())
df['Family_Com_Name'] = df['Family_Com_Name'].apply(lambda x: x.replace('"', '').title())

params = urllib.parse.quote_plus \
(r'Driver={ODBC Driver 17 for SQL Server};Server=tcp:ruffserver.database.windows.net,1433;Database=funData;Uid=connorruff;Pwd=Charlotte99!;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;')
conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
engine = sqlalchemy.create_engine(conn_str, echo=True)

typesDict = {'index': sqlalchemy.INTEGER(),'Species_Code': sqlalchemy.types.NVARCHAR(length=10), 'Species': sqlalchemy.types.NVARCHAR(length=80), 'Scientific_Name': sqlalchemy.types.NVARCHAR(length=80), 'Sci_Name_Codes': sqlalchemy.types.NVARCHAR(length=30), 'Order': sqlalchemy.types.NVARCHAR(length=80), 'Family_Com_Name': sqlalchemy.types.NVARCHAR(length=80), 'Family_Sci_Name': sqlalchemy.types.NVARCHAR(length=80), 'Family_Code': sqlalchemy.types.NVARCHAR(length=10)}
df.to_sql('TAXONOMY', con=engine, schema='REF', if_exists='replace', index=True, dtype=typesDict)
    

2389075
2022-01-04 13:30:04,942 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2022-01-04 13:30:04,944 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-04 13:30:04,988 INFO sqlalchemy.engine.Engine SELECT schema_name()
2022-01-04 13:30:04,991 INFO sqlalchemy.engine.Engine [generated in 0.00277s] ()
2022-01-04 13:30:05,072 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2022-01-04 13:30:05,073 INFO sqlalchemy.engine.Engine [generated in 0.00134s] ()
2022-01-04 13:30:05,141 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2022-01-04 13:30:05,143 INFO sqlalchemy.engine.Engine [generated in 0.00173s] ('BASE TABLE', 'TAXONOMY', 'R

Test: Insert Values Into A Table in the DB

In [41]:

conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};Server=tcp:ruffserver.database.windows.net,1433;Database=funData;Uid=connorruff;Pwd=Charlotte99!;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;')
cursor = conn.cursor()
listOVals = [[1, 2], [3, 4], [5, 6]]
query_string = 'INSERT INTO TMP.test_tabl (first_val, second_val) VALUES (?, ?)'
cursor.executemany(query_string, listOVals)
resp = conn.commit()

print(resp)

None


Export SpeciesCode Data From REF table into a file (file was copied into the repo afterwards)

In [52]:
params = urllib.parse.quote_plus \
(r'Driver={ODBC Driver 17 for SQL Server};Server=tcp:ruffserver.database.windows.net,1433;Database=funData;Uid=connorruff;Pwd=Charlotte99!;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;')
conn_str = 'mssql+pyodbc:///?odbc_connect={}'.format(params)
engine = sqlalchemy.create_engine(conn_str, echo=True)

df = pd.read_sql('SELECT Species_Code, Species FROM REF.TAXONOMY', engine)
df.to_json('C:/Users/conno/Desktop/TMP/Taxonomy.json', orient='split')


2022-01-04 17:36:31,048 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2022-01-04 17:36:31,053 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-04 17:36:31,092 INFO sqlalchemy.engine.Engine SELECT schema_name()
2022-01-04 17:36:31,095 INFO sqlalchemy.engine.Engine [generated in 0.00346s] ()
2022-01-04 17:36:31,177 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2022-01-04 17:36:31,180 INFO sqlalchemy.engine.Engine [generated in 0.00368s] ()
2022-01-04 17:36:31,522 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE [INFORMATION_SCHEMA].[TABLES].[TABLE_TYPE] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] = CAST(? AS NVARCHAR(max)) AND [INFORMATION_SCHEMA].[TABLES].[TABLE_SCHEMA] = CAST(? AS NVARCHAR(max))
2022-01-04 17:36:31,525 INFO sqlalchemy.engine.Engine [generated in 0.00316s] ('BASE TABLE', 'SELECT Species_Code, 